<a href="https://www.kaggle.com/code/aisuko/rag-q-a-with-customise-dataset?scriptVersionId=166899821" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

In this notebook, we will use our own dataset as the knwoloedge of the RAG. We will load our data from CSV file. Next, loading `RagRetriever` witht the customise data. Finally, we will do some inference under the context of the customize data.


# Installing faiss(GPU+CPU)

Here we install Faceboook Faiss which is a library for efficient **similarity search** and **cluster of dense vectors**. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Some of the most useful algorithms are implemented on the GPU. Further more, the GPU implementation can accept input from either CPU or GPU memeory. 

In [ ]:
%%capture
!conda install -c pytorch -c nvidia faiss-gpu=1.8.0 -y

In [ ]:
%%capture
!pip install transformers==4.38.2
# !pip install accelerate==0.27.2
!pip install datasets==2.18.0 # Fix issue numpy attributes error of RagRetriever
# !pip install peft==0.9.0
# !pip install bitsandbytes==0.42.0

In [ ]:
import os
import torch
import faiss # for checking faiss-gpu
import warnings

os.environ['CSV_PATH']='/kaggle/input/knowledge-dataset/own_knwoledge dataset.csv'
os.environ['CSV_DEMO']='/kaggle/input/knowledge-dataset/my_knowledge_dataset.csv'
os.environ['EM_MODEL']='facebook/dpr-ctx_encoder-multiset-base'
os.environ['RAG_MODEL']='facebook/rag-sequence-nq'

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic=True
    # https://github.com/huggingface/transformers/issues/28731
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    device='cuda'
else:
    device='cpu'
    
warnings.filterwarnings('ignore')

print(device)

# Checking the Basic Information of Data

The dataset needed for RAG must have two columns:
- title(string): title of the document
- text(string): text of a passage of the document

We visualization the data and make sure it's format was corrected.

In [ ]:
# Bad case
import pandas as pd

df=pd.read_csv(os.getenv('CSV_PATH'))
print(df.shape)
df.head()

In [ ]:

df1=pd.read_csv(os.getenv("CSV_DEMO"))
print(df1.shape)
df1.head()

# Loading the Data from CSV

We load the CSV file and split the data into passages of 100 words

In [ ]:
from datasets import Dataset, load_dataset
from typing import List, Optional


def split_text(text: str, n=100, character=" ") -> List[str]:
    """Split the text every ``n``-th occurrence of ``character``"""
    text = text.split(character)
    return [character.join(text[i : i + n]).strip() for i in range(0, len(text), n)]


def split_documents(documents: dict) -> dict:
    """Split documents into passages"""
    titles, texts = [], []
    for title, text in zip(documents["title"], documents["text"]):
        if text is not None:
            for passage in split_text(text):
                titles.append(title if title is not None else "")
                texts.append(passage)
    return {"title": titles, "text": texts}


# Using pandas dataframe without set columnes names will cause issue
# dataset=Dataset.from_pandas(df, split="train")
# dataset=dataset.map(split_documents, batched=True, num_proc=4) # 4 vCPUs in Kaggle

# Spliting the documents into passages of 100 words
dataset = load_dataset("csv", data_files=[os.getenv('CSV_DEMO')], split="train", delimiter="\t", column_names=["title", "text"])
dataset

# Create Dataset from CSV



In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast

ctx_encoder=DPRContextEncoder.from_pretrained(os.getenv("EM_MODEL")).to(device)
ctx_tokenizer=DPRContextEncoderTokenizerFast.from_pretrained(os.getenv("EM_MODEL"))

In [ ]:
from functools import partial
from datasets import Features, Sequence, Value


def embed(documents: dict, ctx_encoder: DPRContextEncoder, ctx_tokenizer: DPRContextEncoderTokenizerFast) -> dict:
    """Compute the DPR embeddings of document passages"""
    print(documents["title"])
    print(documents["text"])
    input_ids = ctx_tokenizer(documents["title"], documents["text"], truncation=True, padding="longest", return_tensors="pt")["input_ids"]
    embeddings = ctx_encoder(input_ids.to(device=device), return_dict=True).pooler_output
    return {"embeddings": embeddings.detach().cpu().numpy()}
    
new_features = Features({
    "text": Value("string"), 
    "title": Value("string"), 
    "embeddings": Sequence(Value("float32"))})  # optional, save as float32 instead of float64 to save space


dataset = dataset.map(
    partial(embed, ctx_encoder=ctx_encoder, ctx_tokenizer=ctx_tokenizer),
    batched=True,
    batch_size=16,
    features=new_features,
)

dataset.save_to_disk('my_knowledge_dataset')

We can also load the dataset from the local disk.

```python
from datasets import load_from_disk

dataset=load_from_disk(path_passages)
```

# Index the Dataset

We are going to use the Faiss implementation of HNSW for fast approcimate nearest neighbor search.

In [ ]:
dimension=768 # The dimension of the embeddings to pass to the HNSW Faiss index.
m=128 # The number of bi-directional links created for every new element during the HNSW index construction.

index=faiss.IndexHNSWFlat(dimension, m, faiss.METRIC_INNER_PRODUCT)
type(index)

In [ ]:
dataset.add_faiss_index("embeddings", custom_index=index)

# Save the Index

In [ ]:
dataset.get_index("embeddings").save("my_knowledge_hnsw_index.faiss")

# Loading RAG

We load RagRetriever and RagSequenceForGeneration seperately.

In [ ]:
from transformers import RagRetriever, RagSequenceForGeneration, RagTokenizer

retriever=RagRetriever.from_pretrained(os.getenv("RAG_MODEL"), index="custom", indexed_dataset=dataset)

In [ ]:
model=RagSequenceForGeneration.from_pretrained(os.getenv("RAG_MODEL"), retriever=retriever).to(device)

In [ ]:
tokenizer=RagTokenizer.from_pretrained(os.getenv("RAG_MODEL"))

# Infernece

In [ ]:
question="What does Moses' rod turn into?"

def inference(question:str):
    input_ids=tokenizer.question_encoder(question, return_tensors="pt").to(device)
    generated=model.generate(input_idsp['input_ids'])
    generated_str=tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
    return generated_str

inference(question)

In [ ]:
question2="How many people live in Melbourne?"
inference(question2)

In [ ]:
question3="What is the relationship between Nintendo and Pokémon?"
inference(question3)

In [ ]:
question4="What does Moses' rod turn into?"
inference(question4)

In [ ]:
question5="Where is Pokémon company?"
inference(question5)

# Conclusion

As we can see if we want to use our own dataset. We need to make sure our private data can cover same basic knowledge around the world. Otherwise, The answer might be worse than we assumed.